# Austin Animal Shelter Analysis -- Data Cleaning and Feature Engineering

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import joblib
import time

In [2]:
sns.set_style("darkgrid")
sns.set(font_scale=2)

In [3]:
df_out = pd.read_csv('Austin_Animal_Center_Outcomes.csv')
df_in = pd.read_csv('Austin_Animal_Center_Intakes.csv')

In [4]:
df_out.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,03/18/2014 11:47:00 AM,Mar 2014,03/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


In [5]:
df_in.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
1,A665739,*Alana,10/22/2013 11:11:00 AM,October 2013,Austin (TX),Stray,Normal,Cat,Intact Female,1 month,Domestic Medium Hair Mix,Black
2,A665763,NaN,10/22/2013 03:10:00 PM,October 2013,E Riverside Dr/Royal Crest Dr in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Cairn Terrier Mix,Tan/White
3,A379998,Disciple,10/23/2013 11:42:00 AM,October 2013,51St And Grover in Austin (TX),Stray,Normal,Dog,Intact Male,10 years,Pit Bull,Black
4,A634503,Otter,10/01/2013 02:49:00 PM,October 2013,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,2 years,Norfolk Terrier Mix,Tan


## General cleanup

First, perform general cleanup: convert the DateTime columns to datetime objects and remove the Name columns.

In [6]:
df_out = df_out.drop('Name', axis=1)
df_in = df_in.drop('Name', axis=1)
df_out['DateTime'] = pd.to_datetime(df_out['DateTime'], format="%m/%d/%Y %I:%M:%S %p")
df_in['DateTime'] = pd.to_datetime(df_in['DateTime'], format="%m/%d/%Y %I:%M:%S %p")

/var/folders/qp/8frj290531z81h608x0zwkk80000gn/T/ipykernel_94857/2653004761.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_out['DateTime'] = pd.to_datetime(df_out['DateTime'])


Continue general cleanup: convert unwieldy column names to something easier to use

In [7]:
rename_dict_in = {'Animal ID':'id', 'Found Location':'intake_loc', 'DateTime':'intake_time',
                  'Intake Type':'intake_type', 'Intake Condition':'intake_condition', 'Animal Type':'animal_type',
                 'Sex upon Intake':'intake_sex', 'Age upon Intake':'intake_age','Breed':'breed','Color':'color'}
rename_dict_out = {'Animal ID':'id', 'Date of Birth':'dob', 'DateTime':'outcome_time',
                  'Outcome Type':'outcome_type', 'Outcome Subtype':'outcome_subtype',
                'Animal Type':'animal_type','Sex upon Outcome':'outcome_sex', 
                   'Age upon Outcome':'outcome_age','Breed':'breed','Color':'color'}
df_in=df_in.rename(rename_dict_in,axis=1)
df_out=df_out.rename(rename_dict_out,axis=1)

## Merge dataframes

Now, we want to merge these two dataframes. However, there are animals who are repeat visitors to the shelter. To handle this, we create a new column that we can merge upon. This new column will hold unique visit IDs combining the animal ID and the number of visitations, e.g., an animal with Animal ID A521520 that visits three times has three rows in the dataframes, each will have different visit IDs: A5215200, A5215201, and A5215202.

First, let's sort these dataframes by time

In [8]:
df_in = df_in.sort_values(by='intake_time').reset_index(drop=True)
df_out = df_out.sort_values(by='outcome_time').reset_index(drop=True)

In [9]:
df_in['repeat'] = df_in['id'].duplicated(keep=False)
df_out['repeat'] = df_out['id'].duplicated(keep=False)

In [10]:
df_in['repeat'].value_counts()

repeat
False    121422
True      27485
Name: count, dtype: int64

In [11]:
df_out['repeat'].value_counts()

repeat
False    121609
True      27466
Name: count, dtype: int64

In [12]:
def find_dup(df, col): 
    # modified from https://stackoverflow.com/questions/46629518/find-indices-of-duplicate-rows-in-pandas-dataframe
    ''' Find indices of rows with duplication in column col. 
    if you just want to find duplicated rows, then you can replace col with df.columns.tolist() instead.'''
    df_dupes = df[df[col].duplicated(keep=False)]
    dupe_list = df_dupes.groupby(col).apply(lambda x: tuple(x.index)).tolist()
    return dupe_list

def add_mult_col(df, dupe_list):
    ''' Add a column to df labelling the 1st, 2nd, 3rd etc instance of duplications '''
    df['mult'] = 0
    for dupes in dupe_list:
        dupe_count = 0
        for dupe in dupes: 
            df.loc[dupe, 'mult'] = dupe_count
            dupe_count += 1

In [13]:
intake_repeated = find_dup(df_in, 'id')
add_mult_col(df_in,intake_repeated)

outcome_repeated = find_dup(df_out, 'id')
add_mult_col(df_out,outcome_repeated)

In [14]:
# Create a unique visit ID column combining the animal ID and the number of visitations:
df_in['visit_id'] = df_in['id'] + df_in['mult'].astype(str)
df_out['visit_id'] = df_out['id'] + df_out['mult'].astype(str)

Now let us merge the two dataframes:

In [15]:
shelter_df = df_in.merge(df_out, how='inner', on='visit_id')

We are using an inner join to only keep animals that are both in df_in and df_out. We lose some animals these way. For example, there might be animals that are still in the shelter when the data was taken!

In [16]:
print('Length of df_in is:', len(df_in))
print('Length of df_out is:', len(df_out))
print('Length of shelter_df is:', len(shelter_df))

Length of df_in is: 148907
Length of df_out is: 149075
Length of shelter_df is: 148150


Let us clean up the columns -- we ended up with some duplicated columns.

In [17]:
duplicated_columns = ['id_y', 'animal_type_y', 'breed_y', 'color_y', 'repeat_y', 'mult_y']
rename_dict = {'id_x':'id', 'MonthYear_x':'intake_MonthYear','animal_type_x':'animal_type',
               'breed_x':'breed','color_x':'color','repeat_x':'repeat','mult_x':'mult',
               'MonthYear_y':'outcome_MonthYear'}
shelter_df = shelter_df.drop(duplicated_columns, axis=1)
shelter_df = shelter_df.rename(rename_dict, axis=1)

In [18]:
shelter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148150 entries, 0 to 148149
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 148150 non-null  object        
 1   intake_time        148150 non-null  datetime64[ns]
 2   intake_MonthYear   148150 non-null  object        
 3   intake_loc         148150 non-null  object        
 4   intake_type        148150 non-null  object        
 5   intake_condition   148150 non-null  object        
 6   animal_type        148150 non-null  object        
 7   intake_sex         148148 non-null  object        
 8   intake_age         148149 non-null  object        
 9   breed              148150 non-null  object        
 10  color              148150 non-null  object        
 11  repeat             148150 non-null  bool          
 12  mult               148150 non-null  int64         
 13  visit_id           148150 non-null  object  

## Dropping and imputing missing data

There is a lot of null values in outcome_subtype. This column contains miscellaneous information such as whether the animal is a 'Rabies Risk' or is currently 'At Vet':

In [19]:
shelter_df['outcome_subtype'].unique()

array(['Partner', nan, 'Foster', 'SCRP', 'In Foster', 'Suffering',
       'Medical', 'Aggressive', 'Court/Investigation', 'Rabies Risk',
       'In Kennel', 'Behavior', 'Offsite', 'Possible Theft', 'Enroute',
       'In Surgery', 'At Vet', 'Barn', 'Underage', 'Snr', 'Field',
       'Customer S', 'Prc', 'Out State', 'Emer', 'Emergency', 'In State'],
      dtype=object)

While this column contains interesting information, we will not explore it for now. Let's drop this column:

In [20]:
shelter_df = shelter_df.drop('outcome_subtype', axis=1)

For income_sex and outcome_sex, some of the NaN values are recorded as 'Unknown'. 

In [21]:
shelter_df = shelter_df.replace('Unknown',np.nan)

In [22]:
shelter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148150 entries, 0 to 148149
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 148150 non-null  object        
 1   intake_time        148150 non-null  datetime64[ns]
 2   intake_MonthYear   148150 non-null  object        
 3   intake_loc         148150 non-null  object        
 4   intake_type        148150 non-null  object        
 5   intake_condition   148146 non-null  object        
 6   animal_type        148150 non-null  object        
 7   intake_sex         136541 non-null  object        
 8   intake_age         148149 non-null  object        
 9   breed              148147 non-null  object        
 10  color              148149 non-null  object        
 11  repeat             148150 non-null  bool          
 12  mult               148150 non-null  int64         
 13  visit_id           148150 non-null  object  

There are rows with missing income_sex, outcome_sex, income_age, outcome_type, intake_condition, breed, and outcome_age. We will drop rows with missing values on these columns except for outcome_age -- they represent a small percentage of the total data, so their removal should not affect the conclusion of our analysis. 

In [23]:
shelter_df = shelter_df.dropna(subset=['intake_sex','outcome_sex','intake_age','outcome_type', 'intake_condition', 'breed'])

We have a few rows with missing outcome_age. We can compute this based on the intake_age and the length of stay in the shelter! First, let's add another feature recording the length of stay at the shelter in days. This feature might be useful later too.

In [24]:
def get_frac_day_dif(row):
    delta = row['stay_length']
    return delta.total_seconds() / timedelta(days=1).total_seconds()

shelter_df['stay_length'] = shelter_df.loc[:,'outcome_time'] - shelter_df.loc[:,'intake_time']
shelter_df['stay_length_days'] = shelter_df.apply(get_frac_day_dif, axis=1)

Next, let us turn the intake_age into years. While we're at it, let's convert outcome_age into years as well. 

In [25]:
def convert_age(row, col):
    ''' Convert age to years '''
    res = np.nan
    if pd.isnull(row[col]):
        pass
    else:
        if 'year' in row[col]:
            res = float(row[col].rstrip(' years'))
        elif 'month' in row[col]:
            res = float(row[col].rstrip(' months'))*1/12
        elif 'week' in row[col]:
            res = float(row[col].rstrip(' weeks'))*1/52
        elif 'day' in row[col]:
            res = float(row[col].rstrip(' days'))*1/365
    return res

shelter_df['intake_age_years'] = shelter_df.apply(lambda x: convert_age(x,'intake_age'), axis=1)
shelter_df['outcome_age_years'] = shelter_df.apply(lambda x: convert_age(x,'outcome_age'), axis=1)

In [26]:
shelter_df

,id,intake_time,intake_MonthYear,intake_loc,intake_type,intake_condition,animal_type,intake_sex,intake_age,breed,...,outcome_time,outcome_MonthYear,dob,outcome_type,outcome_sex,outcome_age,stay_length,stay_length_days,intake_age_years,outcome_age_years
0,A664272,2013-10-01 01:12:00,October 2013,12005 Garden Grove Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,1 year,Cairn Terrier Mix,...,2013-11-17 11:01:00,Nov 2013,10/01/2012,Transfer,Spayed Female,1 year,47 days 09:49:00,47.409028,1.000000,1.000000
1,A664273,2013-10-01 01:32:00,October 2013,Berkman Dr & Briarcliff Blvd in Austin (TX),Stray,Normal,Dog,Intact Female,1 year,Dachshund/Chihuahua Shorthair,...,2013-10-13 15:22:00,Oct 2013,04/01/2012,Adoption,Spayed Female,1 year,12 days 13:50:00,12.576389,1.000000,1.000000
2,A639030,2013-10-01 01:36:00,October 2013,46Th And Ave G in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,German Shepherd Mix,...,2013-10-02 17:08:00,Oct 2013,10/08/2005,Return to Owner,Spayed Female,7 years,1 days 15:32:00,1.647222,7.000000,7.000000
3,A664253,2013-10-01 01:44:00,October 2013,Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,Black/Tan Hound Mix,...,2013-10-07 14:15:00,Oct 2013,10/01/2011,Return to Owner,Neutered Male,2 years,6 days 12:31:00,6.521528,2.000000,2.000000
4,A658012,2013-10-01 01:48:00,October 2013,1505 Darold Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,3 years,Chihuahua Shorthair Mix,...,2013-10-01 18:27:00,Oct 2013,07/05/2010,Return to Owner,Neutered Male,3 years,0 days 16:39:00,0.693750,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148144,A876245,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,2 days,Domestic Shorthair,...,2023-03-11 18:38:00,Mar 2023,03/09/2023,Transfer,Intact Male,2 days,0 days 13:04:00,0.544444,0.005479,0.005479
148145,A876243,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Female,2 days,Domestic Shorthair,...,2023-03-11 18:39:00,Mar 2023,03/09/2023,Transfer,Intact Female,2 days,0 days 13:05:00,0.545139,0.005479,0.005479
148146,A876242,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,2 days,Domestic Shorthair,...,2023-03-11 18:39:00,Mar 2023,03/09/2023,Transfer,Intact Male,2 days,0 days 13:05:00,0.545139,0.005479,0.005479
148147,A876244,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,2 days,Domestic Shorthair,...,2023-03-11 18:38:00,Mar 2023,03/09/2023,Transfer,Intact Male,2 days,0 days 13:04:00,0.544444,0.005479,0.005479


In [27]:
shelter_df['outcome_age_years'] = shelter_df.apply(
    lambda row: row['intake_age_years']+row['stay_length_days']/365 if pd.isnull(row['outcome_age']) else row['outcome_age_years'],
    axis=1
)

In [28]:
shelter_df

,id,intake_time,intake_MonthYear,intake_loc,intake_type,intake_condition,animal_type,intake_sex,intake_age,breed,...,outcome_time,outcome_MonthYear,dob,outcome_type,outcome_sex,outcome_age,stay_length,stay_length_days,intake_age_years,outcome_age_years
0,A664272,2013-10-01 01:12:00,October 2013,12005 Garden Grove Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,1 year,Cairn Terrier Mix,...,2013-11-17 11:01:00,Nov 2013,10/01/2012,Transfer,Spayed Female,1 year,47 days 09:49:00,47.409028,1.000000,1.000000
1,A664273,2013-10-01 01:32:00,October 2013,Berkman Dr & Briarcliff Blvd in Austin (TX),Stray,Normal,Dog,Intact Female,1 year,Dachshund/Chihuahua Shorthair,...,2013-10-13 15:22:00,Oct 2013,04/01/2012,Adoption,Spayed Female,1 year,12 days 13:50:00,12.576389,1.000000,1.000000
2,A639030,2013-10-01 01:36:00,October 2013,46Th And Ave G in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,German Shepherd Mix,...,2013-10-02 17:08:00,Oct 2013,10/08/2005,Return to Owner,Spayed Female,7 years,1 days 15:32:00,1.647222,7.000000,7.000000
3,A664253,2013-10-01 01:44:00,October 2013,Austin (TX),Public Assist,Normal,Dog,Neutered Male,2 years,Black/Tan Hound Mix,...,2013-10-07 14:15:00,Oct 2013,10/01/2011,Return to Owner,Neutered Male,2 years,6 days 12:31:00,6.521528,2.000000,2.000000
4,A658012,2013-10-01 01:48:00,October 2013,1505 Darold Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,3 years,Chihuahua Shorthair Mix,...,2013-10-01 18:27:00,Oct 2013,07/05/2010,Return to Owner,Neutered Male,3 years,0 days 16:39:00,0.693750,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148144,A876245,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,2 days,Domestic Shorthair,...,2023-03-11 18:38:00,Mar 2023,03/09/2023,Transfer,Intact Male,2 days,0 days 13:04:00,0.544444,0.005479,0.005479
148145,A876243,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Female,2 days,Domestic Shorthair,...,2023-03-11 18:39:00,Mar 2023,03/09/2023,Transfer,Intact Female,2 days,0 days 13:05:00,0.545139,0.005479,0.005479
148146,A876242,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,2 days,Domestic Shorthair,...,2023-03-11 18:39:00,Mar 2023,03/09/2023,Transfer,Intact Male,2 days,0 days 13:05:00,0.545139,0.005479,0.005479
148147,A876244,2023-03-11 05:34:00,March 2023,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,2 days,Domestic Shorthair,...,2023-03-11 18:38:00,Mar 2023,03/09/2023,Transfer,Intact Male,2 days,0 days 13:04:00,0.544444,0.005479,0.005479


We are left with a lot of duplicated information, let us clean them up

In [29]:
duplicated_info_col = ['intake_MonthYear', 'outcome_MonthYear', 'intake_age', 'outcome_age', 'stay_length']
shelter_df = shelter_df.drop(duplicated_info_col, axis=1)

In [30]:
shelter_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136516 entries, 0 to 148148
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 136516 non-null  object        
 1   intake_time        136516 non-null  datetime64[ns]
 2   intake_loc         136516 non-null  object        
 3   intake_type        136516 non-null  object        
 4   intake_condition   136516 non-null  object        
 5   animal_type        136516 non-null  object        
 6   intake_sex         136516 non-null  object        
 7   breed              136516 non-null  object        
 8   color              136516 non-null  object        
 9   repeat             136516 non-null  bool          
 10  mult               136516 non-null  int64         
 11  visit_id           136516 non-null  object        
 12  outcome_time       136516 non-null  datetime64[ns]
 13  dob                136516 non-null  object       

Note that: with the intake_age_years and outcome_age_years columns: the ages are recorded in integer units of years, weeks, days, or months. This leads to some ambiguity as we are not sure what the rounding convention is in recording the dataset: 0 years can mean 0-365 days.

The 'stay_length_days' column also shows that there are a few rows with <0 length of stay. There must be a mistake in these entries, and due to their very small number count, we will simply drop them from the dataset. 

In [31]:
shelter_df = shelter_df[shelter_df['stay_length_days']>0]

In [32]:
shelter_df

,id,intake_time,intake_loc,intake_type,intake_condition,animal_type,intake_sex,breed,color,repeat,mult,visit_id,outcome_time,dob,outcome_type,outcome_sex,stay_length_days,intake_age_years,outcome_age_years
0,A664272,2013-10-01 01:12:00,12005 Garden Grove Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,Cairn Terrier Mix,Brown,False,0,A6642720,2013-11-17 11:01:00,10/01/2012,Transfer,Spayed Female,47.409028,1.000000,1.000000
1,A664273,2013-10-01 01:32:00,Berkman Dr & Briarcliff Blvd in Austin (TX),Stray,Normal,Dog,Intact Female,Dachshund/Chihuahua Shorthair,Brown,False,0,A6642730,2013-10-13 15:22:00,04/01/2012,Adoption,Spayed Female,12.576389,1.000000,1.000000
2,A639030,2013-10-01 01:36:00,46Th And Ave G in Austin (TX),Stray,Normal,Dog,Spayed Female,German Shepherd Mix,Tan,True,0,A6390300,2013-10-02 17:08:00,10/08/2005,Return to Owner,Spayed Female,1.647222,7.000000,7.000000
3,A664253,2013-10-01 01:44:00,Austin (TX),Public Assist,Normal,Dog,Neutered Male,Black/Tan Hound Mix,Black/Brown,False,0,A6642530,2013-10-07 14:15:00,10/01/2011,Return to Owner,Neutered Male,6.521528,2.000000,2.000000
4,A658012,2013-10-01 01:48:00,1505 Darold Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,Chihuahua Shorthair Mix,Red,False,0,A6580120,2013-10-01 18:27:00,07/05/2010,Return to Owner,Neutered Male,0.693750,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148144,A876245,2023-03-11 05:34:00,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,Domestic Shorthair,White/Black,False,0,A8762450,2023-03-11 18:38:00,03/09/2023,Transfer,Intact Male,0.544444,0.005479,0.005479
148145,A876243,2023-03-11 05:34:00,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Female,Domestic Shorthair,Black/White,False,0,A8762430,2023-03-11 18:39:00,03/09/2023,Transfer,Intact Female,0.545139,0.005479,0.005479
148146,A876242,2023-03-11 05:34:00,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,Domestic Shorthair,Black,False,0,A8762420,2023-03-11 18:39:00,03/09/2023,Transfer,Intact Male,0.545139,0.005479,0.005479
148147,A876244,2023-03-11 05:34:00,9805 Aberdeen Way in Austin (TX),Stray,Neonatal,Cat,Intact Male,Domestic Shorthair,Black/White,False,0,A8762440,2023-03-11 18:38:00,03/09/2023,Transfer,Intact Male,0.544444,0.005479,0.005479


Next, in this project we want to focus just on dogs and cats, so we will remove the non-dog/cat rows. There are not many of them:

In [33]:
shelter_df['animal_type'].value_counts()

animal_type
Dog          82196
Cat          52064
Other         1446
Bird           332
Livestock       16
Name: count, dtype: int64

In [34]:
shelter_df = shelter_df[(shelter_df['animal_type'] == 'Dog') | (shelter_df['animal_type'] == 'Cat')]

Let us now focus on the breed column:

In [35]:
shelter_df['breed'].value_counts()

breed
Domestic Shorthair Mix            29775
Domestic Shorthair                12891
Pit Bull Mix                       9199
Labrador Retriever Mix             7723
Chihuahua Shorthair Mix            6579
                                  ...  
Shetland Sheepdog/Basenji             1
Rat Terrier/Cairn Terrier             1
Queensland Heeler/Great Dane          1
Pit Bull/American Foxhound            1
Rat Terrier/Dachshund Wirehair        1
Name: count, Length: 2595, dtype: int64

Most of the breeds have just a few counts:

In [36]:
row_counts = shelter_df['breed'].value_counts() # get the counts for each class
idx_lt_100 = row_counts[row_counts.lt(100)].index # classes with <100 counts
print('Total number of breeds is', len(row_counts))
print('The number of breeds with less than 100 members is', len(idx_lt_100))

Total number of breeds is 2595
The number of breeds with less than 100 members is 2477


We want to consolidate breeds that have only a few entries into a new class, called 'Others'. 

In [37]:
shelter_df.loc[shelter_df['breed'].isin(idx_lt_100),'breed'] = 'Others'
print('Number of unique breeds is now:', len(shelter_df['breed'].unique()))

Number of unique breeds is now: 119


We can do the same thing on the color column:

In [38]:
shelter_df['color'].value_counts()

color
Black/White          14413
Black                10812
Brown Tabby           7846
White                 4831
Brown/White           4713
                     ...  
Blue Point/Cream         1
Calico Point/Gray        1
Seal Point/Cream         1
Black Tabby/Gray         1
Buff/Blue Merle          1
Name: count, Length: 580, dtype: int64

In [39]:
row_counts = shelter_df['color'].value_counts() # get the counts for each class
idx_lt_100 = row_counts[row_counts.lt(100)].index # classes with <100 counts
print('Total number of color is', len(row_counts))
print('The number of color with less than 100 members is', len(idx_lt_100))

Total number of color is 580
The number of color with less than 100 members is 486


Again we want to consolidate colors that have only a few entries into a new class, called 'Others'. 

In [40]:
shelter_df.loc[shelter_df['color'].isin(idx_lt_100),'color'] = 'Others'
print('Number of unique colors is now:', len(shelter_df['color'].unique()))

Number of unique colors is now: 95


## Feature engineering

In the previous step, we already created two new features that might be useful in the exploration and prediction steps: 'stay_length_days', the length of stay in the shelter and 'mult', the number of times the same animal visits the shelter. Let's see if we can engineer any other features that could be useful!

People often get pets at certain months of the year (e.g., Christmas) or at certain days of the week (e.g., weekends), so we can hypothize that the month or day of the week of outcome_time would be important predictors of an animal's outcome. However, outcome_time is not a good variable for predicting whether an animal will get adopted, as the outcome_time is recorded AFTER an animal exits the shelter. Instead, we want to know the chances for an animal to be adopted when they are still in the shelter!

If we assume that there is some correlation between outcome_time and intake_time, we can instead hypothize that the month or day of the week of intake_time would be important predictors of an animal's outcome.

In [41]:
with pd.option_context('mode.chained_assignment', None):
    shelter_df.loc[:,'intake_day'] = \
                 shelter_df.loc[:,'intake_time'].dt.dayofweek + 1 # +1 so we don't have 0th day of the week
    shelter_df.loc[:,'intake_month'] = shelter_df.loc[:,'intake_time'].dt.month

The outcome_type column record information about what happened to the animals after their visit to the shelter:

In [42]:
shelter_df['outcome_type'].value_counts()

outcome_type
Adoption           67781
Transfer           37121
Return to Owner    23475
Euthanasia          3754
Rto-Adopt            996
Died                 961
Disposal             100
Missing               66
Relocate               4
Stolen                 2
Name: count, dtype: int64

Instead of trying to classify the outcome_type exactly, let us instead try to predict whether a particular animal's visit ended up with a good, bad, or neutral outcome. To do this, let us group the rows with good outcomes (Return to Owner, Adoption, or Rto-Adopt), bad outcomes (Euthanasia, Died, Missing, Disposal), and neutral outcomes (Transfer, Relocate).

In [43]:
good_mask = ((shelter_df['outcome_type']=='Return to Owner')
        | (shelter_df['outcome_type']=='Adoption')
        | (shelter_df['outcome_type']=='Rto-Adopt'))
bad_mask = ((shelter_df['outcome_type']=='Euthanasia')
        | (shelter_df['outcome_type']=='Died')
        | (shelter_df['outcome_type']=='Missing')
        | (shelter_df['outcome_type']=='Disposal')
        | (shelter_df['outcome_type']=='Stolen'))
neutral_mask = ((shelter_df['outcome_type']=='Transfer')
        | (shelter_df['outcome_type']=='Relocate'))

In [44]:
with pd.option_context('mode.chained_assignment', None):
    shelter_df.loc[good_mask, 'outcome'] = 'good'
    shelter_df.loc[neutral_mask, 'outcome'] = 'neutral'
    shelter_df.loc[bad_mask, 'outcome'] = 'bad'

In [45]:
shelter_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134260 entries, 0 to 148148
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id                 134260 non-null  object        
 1   intake_time        134260 non-null  datetime64[ns]
 2   intake_loc         134260 non-null  object        
 3   intake_type        134260 non-null  object        
 4   intake_condition   134260 non-null  object        
 5   animal_type        134260 non-null  object        
 6   intake_sex         134260 non-null  object        
 7   breed              134260 non-null  object        
 8   color              134260 non-null  object        
 9   repeat             134260 non-null  bool          
 10  mult               134260 non-null  int64         
 11  visit_id           134260 non-null  object        
 12  outcome_time       134260 non-null  datetime64[ns]
 13  dob                134260 non-null  object       

In [46]:
joblib.dump(shelter_df, './shelter_df.pkl') 

['./shelter_df.pkl']